In [1]:
import pandas as pd
import numpy as np

In [12]:
# make entire query asin pair all in training or testing to do ranked precision/recall
data_path="data/"
data_name="us"

In [13]:
train=pd.read_csv(data_path+data_name+"_train.tsv",sep="\t")
test=pd.read_csv(data_path+data_name+"_test.tsv",sep="\t")

In [14]:
all_data=pd.concat([train,test],ignore_index=True)

In [15]:
queries=all_data["keyword"].unique()

In [16]:
len(all_data)/len(queries)

3.8241020476670022

In [106]:
np.random.seed(1)
test_queries=set(np.random.choice(queries,replace=False,size=(len(test)*len(queries))//len(all_data)))

In [107]:
test_data=all_data[[query in test_queries for query in all_data["keyword"]]]

In [108]:
train_data=all_data[[query not in test_queries for query in all_data["keyword"]]]

In [109]:
len(test_data)/len(train_data),len(test)/len(train)

(0.11747138964577657, 0.11019913157410315)

In [110]:
test_data.to_csv(data_path+data_name+"_test.tsv",sep="\t",index=False)
train_data.to_csv(data_path+data_name+"_train.tsv",sep="\t",index=False)

In [ ]:
## madelon dataset

In [4]:
data_path="data/"
data_name="madelon"

In [5]:
train=pd.read_csv(data_path+data_name+"_train.txt",sep=" ",header=None)
test=pd.read_csv(data_path+data_name+"_test.txt",sep=" ",header=None)

In [6]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,491,492,493,494,495,496,497,498,499,500
0,-1,1:485,2:477,3:537,4:479,5:452,6:471,7:491,8:476,9:475,...,491:477,492:481,493:477,494:485,495:511,496:485,497:481,498:479,499:475,500:496
1,-1,1:483,2:458,3:460,4:487,5:587,6:475,7:526,8:479,9:485,...,491:463,492:478,493:487,494:338,495:513,496:486,497:483,498:492,499:510,500:517
2,-1,1:487,2:542,3:499,4:468,5:448,6:471,7:442,8:478,9:480,...,491:487,492:481,493:492,494:650,495:506,496:501,497:480,498:489,499:499,500:498
3,1,1:480,2:491,3:510,4:485,5:495,6:472,7:417,8:474,9:502,...,491:491,492:480,493:474,494:572,495:454,496:469,497:475,498:482,499:494,500:461
4,1,1:484,2:502,3:528,4:489,5:466,6:481,7:402,8:478,9:487,...,491:488,492:479,493:452,494:435,495:486,496:508,497:481,498:504,499:495,500:511


In [7]:
train_data=train.iloc[:,1:].apply(lambda col: [int(r.split(":")[1]) for r in col])

In [8]:
train_data[0]=[ int(x>0) for x in train[0]]

In [9]:
test_data=test.iloc[:,1:].apply(lambda col: [int(r.split(":")[1]) for r in col])

In [10]:
test_data[0]=[ int(x>0) for x in test[0]]

In [11]:
test_data.rename(columns={0:"label"},inplace=True)

In [12]:
train_data.rename(columns={0:"label"},inplace=True)

In [15]:
test_data.to_csv(data_path+data_name+"_test.tsv",index=None,sep="\t")
train_data.to_csv(data_path+data_name+"_train.tsv",index=None,sep="\t")

In [ ]:
## gisette_scale data

In [17]:
data_path="data/"
data_name="gisette_scale"

In [18]:
train=pd.read_csv(data_path+data_name+"_train.txt",sep=" ",header=None)
test=pd.read_csv(data_path+data_name+"_test.txt",sep=" ",header=None)

In [80]:
train_data=train.apply(lambda col: [ float(r.split(":")[1]) if type(r)==str and ":" in r else r for r in col])

In [84]:
train_data[0]=[ int(x>0) for x in train_data[0]]

In [85]:
test_data=test.apply(lambda col: [ float(r.split(":")[1]) if type(r)==str and ":" in r else r for r in col])

In [86]:
test_data[0]=[ int(x>0) for x in test_data[0]]

In [92]:
# test_data has more columns than train_data?

In [93]:
test_data.rename(columns={0:"label"},inplace=True)
train_data.rename(columns={0:"label"},inplace=True)

In [97]:
test_data=test_data.iloc[:,:4957]

In [99]:
test_data.shape,train_data.shape

((1000, 4957), (6000, 4957))

In [100]:
test_data.to_csv(data_path+data_name+"_test.tsv",index=None,sep="\t")
train_data.to_csv(data_path+data_name+"_train.tsv",index=None,sep="\t")

In [1]:
import pandas as pd
import numpy as np
from os import listdir

In [2]:
train="data/gisette_scale_train.tsv"
test="data/gisette_scale_test.tsv"
response_name="label"
folder="result/gisette_scale"

In [3]:
train=pd.read_csv(train,sep="\t")
train_names=list(train.columns.values)
file_names=listdir(folder)

In [6]:
allfeature=set(pd.read_csv(folder+"/allfeature.txt",header=None)[0])

In [12]:
set(str(x) for x in allfeature)

{'1', '2', '3', '4', '5', '6', '7', '8', '9'}

In [15]:
len(us_allfeature)

98

In [22]:
pd.DataFrame(us_allfeature).to_csv("result/us/allfeature.txt",header=None,index=None)